# Plotting the delivery areas
Using Geopandas 
Using shape file from: https://public.opendatasoft.com/explore/dataset/georef-netherlands-postcode-pc4/table/?refine.prov_name=Groningen&refine.gem_name=Groningen&location=10,53.21082,6.61855&basemap=jawg.light 

In [87]:
import pandas as pd
import geopandas as geopandas
import matplotlib.pyplot as plt
import numpy
import folium
from shapely.geometry import Point

In [ ]:
geopandas.read_file('GEO-data/georef-netherlands-postcode-pc4.shp').head(5)

In [ ]:
gdf_groningen = geopandas.read_file('GEO-data/georef-netherlands-postcode-pc4.shp')
gdf_groningen

In [109]:
shp_zip_codes = geopandas.read_file("GEO-data/georef-netherlands-postcode-pc4.shp")[["pc4_code", "geometry"]]
#rename column
shp_zip_codes.columns = ["Postcode", "geometry"]
df_dabba = pd.read_excel("Dabba.xlsx", names = ["Klanteigenaar", "Aantal", "Adres", "Postcode", "Datum bezorgd"], usecols="B,F,G,H,O")
# choose delivery day of 12 januari 2021
df_dabba = df_dabba[df_dabba["Datum bezorgd"] == "12/01/2021"] #because on this day the most packages got delivered. 

# change the 6 postal code to the first 4 numberecal codes
df_dabba["Postcode"] = df_dabba["Postcode"].apply(lambda x:x[:4])

# count the amount of unqiue postalcodes
df_dabba['Deliveries'] = df_dabba['Postcode'].map(df_dabba['Postcode'].value_counts())

In [113]:
# Filter all the postalcodes in the geoframe based on the postalcodes in the dabba dataframe
shp_zip_codes_sorted_dabba = shp_zip_codes[shp_zip_codes["Postcode"].isin(df_dabba["Postcode"].unique())]
# create area in km2 column
shp_zip_codes_sorted_dabba["Area (km2)"] = shp_zip_codes_sorted_dabba['geometry'].to_crs({'init': 'epsg:23095'}).map(lambda p: p.area / 10**6)

C:\Users\tobia\AppData\Local\Temp\ipykernel_10896\2091110274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_zip_codes_sorted_dabba.drop(shp_zip_codes_sorted_dabba.index[shp_zip_codes_sorted_dabba.Postcode == 9723], inplace=True)
c:\Users\tobia\miniconda3\envs\lastmile\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\tobia\miniconda3\envs\lastmile\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
# merge dabba dataframe with the geoframe
merge = pd.merge(df_dabba[["Datum bezorgd", "Postcode", "Deliveries"]], shp_zip_codes_sorted_dabba, on="Postcode").drop_duplicates()
# calculate the route length approximation per entry
k = 0.92
n = merge["Deliveries"]
A = merge["Area (km2)"]

# Calculate and add Beardwood as new column in km
merge["Beardwood approx"] = k * numpy.sqrt((n * A))

# Results of 12 januari 2021
Now we have an dataframe including the Beardwood approximation per postalcode based on 12th of january 2021. We can plot the following maps.
1. general results map based on Postalcode
2. specific on map based on Beardwood's approximation without the D variable

In [ ]:
# create a new geoframe based on the merge
results = geopandas.GeoDataFrame(merge[["Postcode", "Deliveries", "Area (km2)", "Beardwood approx"]], geometry=merge["geometry"])
# A CRS tells Python how those coordinates relate to places on the Earth
results.set_crs(epsg=4326, inplace=True) 
results.explore(
    column="Postcode",
    tooltip=['Postcode', 'Area (km2)', 'Deliveries', "Beardwood approx"],
    popup=True,
    tiles="CartoDB positron", 
    cmap="Set1", 
    name="Amount of deliveries in Groningen per postalcode area"
) 

In [ ]:
results.explore(
    column="Beardwood approx",
    tooltip=['Postcode', 'Area (km2)', 'Deliveries', "Beardwood approx"],
    popup=True,
    tiles="CartoDB positron", 
    cmap="OrRd", 
    name="Beardwood approximation per postalcode area"
) 

# Influence of the depot location 
Now we have plotted the data and calculated the Beardwood approximation for each postalcode area, we want to add an other factor: depot location. Let start by adding an location marker on the current latitude and longitude position of Cycloon called depot 0.

In [119]:
depots_locations = [Point(6.598278, 53.210898), Point(6.528022935211814, 53.22563730386978), Point(6.574485373877002, 53.246475527023634), Point(6.555842080055761, 53.201872050167395)]

## Calculate D
Now we want to calculate the distance between the marker on the map and the different postal code areas called D. As D is the distance, we want to multiply it by 2 because you cycle from the depot to the delivery area and back. 

In [125]:
i = 0
for depot_location in depots_locations:
    distances = []
    for phane in results.geometry:
        geo_phane = geopandas.GeoSeries(phane)
        geo_phane.set_crs(23095, inplace=True)
        geo_point = geopandas.GeoSeries(depot_location)
        geo_point.set_crs(23095, inplace=True) 
        d = geo_phane.distance(geo_point).values[0] * 100
        distances.append(d)
    distances = numpy.array(distances)
    results["D{0}".format(i)] = distances
    results["T*_depot_{0}".format(i)] = (2*distances) + results["Beardwood approx"]
    i=i+1

## Calculate the average T* based the results with D locations 
Take the average between the T* columns 

In [126]:
results["mean_T*_depot_0_1"] = results[["T*_depot_0", "T*_depot_1"]].mean(axis=1)
results["mean_T*_depot_0_1_2"] = results[["T*_depot_0", "T*_depot_1", "T*_depot_2"]].mean(axis=1)
results["mean_T*_depot_0_1_2_3"] = results[["T*_depot_0", "T*_depot_1", "T*_depot_2", "T*_depot_3"]].mean(axis=1)
# row_len = df_T_results.shape[1]
# for i in range(row_len):
#     print("i: {0} \n i+1: {1}".format(i,i+1))
#     if i > row_len-2:
#         break
#     else:
#         df_T_results["avg_T*_depot_{0}_{1}".format(i,i+1)] = df_T_results[["T*_depot_{0}".format(i), "T*_depot_{0}".format(i+1)]].mean(axis=1)

In [ ]:
results.plot(kind="barh", x="Postcode", y=["mean_T*_depot_0_1", "mean_T*_depot_0_1_2", "mean_T*_depot_0_1_2_3"], figsize=(16, 20), width=0.85) 
plt.title("T* mean per depot per postal code area")

In [ ]:
m = folium.Map(location=[53.21917, 6.56667], zoom_start=12, tiles="CartoDB positron")
i = 0
for depot_location in depots_locations:
    folium.Marker(
        [depot_location.y, depot_location.x], 
        popup="Depot_{0}".format(i)
    ).add_to(m)
    i = i + 1

# use feature groups for multiple depot layers https://www.nagarajbhat.com/post/folium-visualization/

choropleth = folium.Choropleth(
    results, 
    data=results, 
    key_on='feature.properties.Postcode', 
    columns=["Postcode", "mean_T*_depot_0_1_2_3"], 
    fill_color="OrRd",
    fill_opacity = 0.8,
    line_opacity = 1,
    line_weight=1,
    legend_name = "mean T* per postalcode area based on depot 0, 1, 2 & 3 on 12 january 2021",
    name="Average T* of all depots per Postalcode area").add_to(m)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['Postcode', 'Area (km2)', 'Deliveries', "T*_depot_0", "T*_depot_1", "T*_depot_2", "T*_depot_3", "mean_T*_depot_0_1_2_3"], labels=True)
)

folium.LayerControl().add_to(m)
m